# Обучение модели EnlightenGAN для восстановления изображений с эффектом "засветления/затемнения"
Этаот Notebook демонстрирует процесс fine-tuning модели EnlightenGAN на датасете с искаженными изображениями (эффект brightness/contrast). 

Мы используем предобученные веса и адаптируем модель под разрешение 1024x1448. 

Обучение включает комбинированный loss (L1 + perceptual), мониторинг метрик (Loss, PSNR, SSIM), early stopping и сохранение чекпоинтов.

## Шаг 1: Настройка окружения и выбор GPU
Выбираем свободную GPU, настраиваем PyTorch для оптимальной работы с CUDA.

In [ ]:
import os
import subprocess
import torch
from torch.amp import autocast, GradScaler

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Выбор GPU с максимум свободной памяти
result = subprocess.check_output(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"]
).decode().strip()

gpu_list = [tuple(map(int, line.split(', '))) for line in result.splitlines()]
best_idx, max_free = max(gpu_list, key=lambda x: x[1])

os.environ['CUDA_VISIBLE_DEVICES'] = str(best_idx)
print(f"Выбрана физическая GPU {best_idx} — свободно ~{max_free//1024} GiB")

device = torch.device('cuda:0')
torch.cuda.set_device(device)
torch.cuda.empty_cache()

print(f"Работаем на {device} ({torch.cuda.get_device_name(0)})")
print(f"VRAM allocated после инициализации: {torch.cuda.memory_allocated(device)/(1024**3):.1f} GiB")

## Шаг 2: Импорт библиотек и архитектуры модели
Импортируем необходимые библиотеки, переходим в директорию EnlightenGAN для импорта генератора.

In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Переход в папку EnlightenGAN
os.chdir('/home/kudriavtcevroman-10/EnlightenGAN')
from models.networks import define_G
os.chdir('/home/kudriavtcevroman-10')

# Проверка наличия файлов
if os.path.exists('EnlightenGAN/models/networks.py'):
    print("Файл networks.py найден.")
else:
    print("Ошибка: Файл networks.py не найден. Проверь структуру папки EnlightenGAN.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

## Шаг 3: Инициализация модели и perceptual loss
Определяем опции модели, инициализируем генератор с ngf=160, загружаем предобученные веса частично, добавляем perceptual loss на VGG19.

In [ ]:
from torchvision.models import vgg19

# Опции модели
class Opt:
    def __init__(self):
        self.self_attention = False
        self.use_norm = 1
        self.syn_norm = False
        self.use_avgpool = 0
        self.tanh = False
        self.times_residual = False
        self.linear_add = False
        self.latent_threshold = False
        self.latent_norm = False
        self.linear = False
        self.skip = 1.0

opt = Opt()

# Инициализация генератора
gpu_ids = [0] if torch.cuda.is_available() else []
generator = define_G(input_nc=4, output_nc=3, ngf=160, which_model_netG='sid_unet_resize', norm='batch', 
                     use_dropout=False, gpu_ids=gpu_ids, skip=False, opt=opt).to(device)

# Загрузка предобученных весов частично
weights_path = 'pretrained/200_net_G_A.pth'
if os.path.exists(weights_path):
    checkpoint = torch.load(weights_path, map_location=device)
    pretrained_state = checkpoint.get('params', checkpoint.get('state_dict', checkpoint))
    
    model_state = generator.state_dict()
    for k, v in pretrained_state.items():
        if k in model_state and v.size() == model_state[k].size():
            model_state[k] = v
    
    generator.load_state_dict(model_state, strict=False)
    print("Веса загружены частично. Новые слои дообучатся за 3-5 эпох.")
else:
    print("Предобученные веса не найдены. Модель обучится с нуля.")

# Perceptual loss на VGG19
vgg = vgg19(pretrained=True).features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

def perceptual_loss(output, clean):
    def get_features(x):
        return vgg[:23](x)  # conv4_4
    return nn.L1Loss()(get_features(output), get_features(clean))

print(f"Модель готова с ngf=160 и perceptual loss. VRAM: {torch.cuda.memory_allocated(device)/1024:.1f} MiB")

## Шаг 4: Подготовка датасета
Определяем кастомный датасет для distorted (color + gray) / clean изображений, с трансформацией на разрешение 1024x1448.

In [ ]:
# Кастомный трансформер с padding
class PaddedResize:
    def __init__(self, size=(1024, 1448), div_factor=8):
        self.size = size
        self.div_factor = div_factor

    def __call__(self, img):
        img = transforms.Resize(self.size, interpolation=Image.LANCZOS)(img)
        w, h = img.size
        pad_w = (self.div_factor - w % self.div_factor) % self.div_factor
        pad_h = (self.div_factor - h % self.div_factor) % self.div_factor
        if pad_w != 0 or pad_h != 0:
            img = transforms.Pad((0, 0, pad_w, pad_h), fill=0)(img)
        return img

# Трансформации
transform = transforms.Compose([
    PaddedResize(size=(1024, 1448)),
    transforms.ToTensor(),
])

# Кастомный датасет
class BrightnessContrastDataset(Dataset):
    def __init__(self, distorted_dir, clean_dir, transform=None):
        self.distorted_images = sorted(os.listdir(distorted_dir))
        self.clean_images = sorted(os.listdir(clean_dir))
        self.distorted_dir = distorted_dir
        self.clean_dir = clean_dir
        self.transform = transform

    def __len__(self): return len(self.distorted_images)

    def __getitem__(self, idx):
        dist_color = Image.open(os.path.join(self.distorted_dir, self.distorted_images[idx])).convert('RGB')
        dist_gray = dist_color.convert('L')
        clean = Image.open(os.path.join(self.clean_dir, self.clean_images[idx])).convert('RGB')
        if self.transform:
            dist_color = self.transform(dist_color)
            dist_gray = transforms.Compose([transforms.Grayscale(num_output_channels=1), self.transform])(dist_color)
            clean = self.transform(clean)
        return dist_color, dist_gray, clean

# Создание датасетов и лоадеров
train_dataset = BrightnessContrastDataset('brightness_contrast/train/distorted', 'brightness_contrast/train/clean', transform=transform)
val_dataset = BrightnessContrastDataset('brightness_contrast/val/distorted', 'brightness_contrast/val/clean', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Шаг 5: Настройка обучения
Определяем combined loss (L1 + perceptual), метрики, оптимизатор (AdamW), scheduler и early stopping.

In [ ]:
import time
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Комбинированный loss
l1_loss = nn.L1Loss()
def combined_loss(output, clean):
    return 0.7 * l1_loss(output, clean) + 0.3 * perceptual_loss(output, clean)

# Метрики
psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

# Оптимизатор и scaler
optimizer = AdamW(generator.parameters(), lr=2e-4, weight_decay=1e-4)
scaler = GradScaler()

# Scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Early stopping
patience = 10
early_stop_counter = 0
best_val_psnr = -float('inf')

# Директория для чекпоинтов
os.makedirs('checkpoints', exist_ok=True)
checkpoint_path = 'checkpoints/enlightengan_bc_ep{}.pth'

# Списки для метрик
train_losses, val_losses = [], []
train_psnrs, val_psnrs = [], []
train_ssims, val_ssims = [], []

num_epochs = 500

## Шаг 6: Цикл обучения
Обучаем модель с mixed precision, мониторим метрики, сохраняем чекпоинты и применяем early stopping.

In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()
    generator.train()
    train_loss = train_psnr = train_ssim = 0.0

    for distorted_color, distorted_gray, clean in tqdm(train_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [train]"):
        distorted_color = distorted_color.to(device, non_blocking=True)
        distorted_gray = distorted_gray.to(device, non_blocking=True)
        clean = clean.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            output = generator(distorted_color, distorted_gray)
            loss = combined_loss(output, clean)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
        with torch.no_grad():
            train_psnr += psnr_metric(output, clean).item()
            train_ssim += ssim_metric(output, clean).item()

    avg_train_loss = train_loss / len(train_loader)
    avg_train_psnr = train_psnr / len(train_loader)
    avg_train_ssim = train_ssim / len(train_loader)
    train_losses.append(avg_train_loss)
    train_psnrs.append(avg_train_psnr)
    train_ssims.append(avg_train_ssim)

    scheduler.step()

    # Валидация
    generator.eval()
    val_loss = val_psnr = val_ssim = 0.0
    with torch.no_grad():
        for distorted_color, distorted_gray, clean in tqdm(val_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [val]"):
            distorted_color = distorted_color.to(device, non_blocking=True)
            distorted_gray = distorted_gray.to(device, non_blocking=True)
            clean = clean.to(device, non_blocking=True)
            with autocast(device_type='cuda'):
                output = generator(distorted_color, distorted_gray)
                val_loss += combined_loss(output, clean).item()
                val_psnr += psnr_metric(output, clean).item()
                val_ssim += ssim_metric(output, clean).item()

    avg_val_loss = val_loss / len(val_loader)
    avg_val_psnr = val_psnr / len(val_loader)
    avg_val_ssim = val_ssim / len(val_loader)
    val_losses.append(avg_val_loss)
    val_psnrs.append(avg_val_psnr)
    val_ssims.append(avg_val_ssim)

    print(f"\nЭпоха {epoch+1:02d} | Train Loss: {avg_train_loss:.5f} | Val Loss: {avg_val_loss:.5f} | "
          f"Val PSNR: {avg_val_psnr:.2f} dB | Val SSIM: {avg_val_ssim:.4f} | Время: {time.time() - start_time:.2f} с | "
          f"VRAM: {torch.cuda.memory_allocated(device)/1024:.1f} MiB")

    # Сохранение чекпоинта
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': generator.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_psnr': best_val_psnr,
    }, checkpoint_path.format(epoch+1))
    print(f"Чекпоинт сохранён для эпохи {epoch+1}")

    # Early stopping
    if avg_val_psnr > best_val_psnr:
        best_val_psnr = avg_val_psnr
        torch.save(generator.state_dict(), "checkpoints/enlightengan_best_psnr.pth")
        print("    → Лучшая модель сохранена!")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        print(f"    → Нет улучшения ({early_stop_counter}/{patience})")

    if early_stop_counter >= patience:
        print(f"Early stopping после {patience} эпох без улучшения.")
        break

    torch.cuda.empty_cache()

print("Fine-tuning EnlightenGAN завершён")

## Шаг 7: Сохранение финальной модели
Сохраняем обученные веса для дальнейшего использования в инференсе.

In [ ]:
torch.save(generator.state_dict(), 'finetuned_enlightengan.pth')
print("Финальная модель сохранена как finetuned_enlightengan.pth.")

## Шаг 8: Визуализация метрик
Строим графики для анализа сходимости и проверки на переобучение.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

axs[0].plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', marker='o')
axs[0].plot(range(1, len(val_losses)+1), val_losses, label='Val Loss', marker='o')
axs[0].set_title('Loss over Epochs')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(range(1, len(train_psnrs)+1), train_psnrs, label='Train PSNR', marker='o')
axs[1].plot(range(1, len(val_psnrs)+1), val_psnrs, label='Val PSNR', marker='o')
axs[1].set_title('PSNR over Epochs')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('PSNR (dB)')
axs[1].legend()
axs[1].grid(True)

axs[2].plot(range(1, len(train_ssims)+1), train_ssims, label='Train SSIM', marker='o')
axs[2].plot(range(1, len(val_ssims)+1), val_ssims, label='Val SSIM', marker='o')
axs[2].set_title('SSIM over Epochs')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('SSIM')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.savefig('enlightengan_metrics.png')
plt.show()
print("Графики сохранены как enlightengan_metrics.png. Проверьте val на рост (PSNR >30-35 dB, SSIM >0.9 идеально для документов).")